# CamilloH3K27me3

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import joblib
import numpy as np

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.CamilloH3K27me3)

class CamilloH3K27me3(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.CamilloH3K27me3()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'camilloh3k27me3'
model.metadata["data_type"] = 'histone mark'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2023
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "de Lima Camillo, Lucas Paulo, et al. \"Histone mark age of human tissues and cells.\" bioRxiv (2023): 2023-08."
model.metadata["doi"] = 'https://doi.org/10.1101/2023.08.21.554165'
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download GitHub repository

In [5]:
github_url = "https://github.com/rsinghlab/HistoneClocks.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load features

#### From CSV file

In [6]:
histone = 'H3K' + model.metadata["clock_name"].split('k')[1]

feature_selector_path = 'HistoneClocks/results/models/' + histone + '_feature_selector.pkl'
feature_selector = joblib.load(feature_selector_path)

dim_reduction_path = 'HistoneClocks/results/models/' + histone + '_dim_reduction.pkl'
dim_reduction = joblib.load(dim_reduction_path)

ard_model_path = 'HistoneClocks/results/models/' + histone + '_model.pkl'
ard_model = joblib.load(ard_model_path)

genes = pd.read_csv('HistoneClocks/metadata/Ensembl-105-EnsDb-for-Homo-sapiens-genes.csv')
chromosomes = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9', 'X']
genes = genes[genes['chr'].apply(lambda x: x in chromosomes)]
genes.index = genes.gene_id
model.features = genes.gene_id[np.abs(feature_selector.coef_) > 0].tolist()

/Users/lucascamillo/mambaforge/envs/brain/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.0.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/lucascamillo/mambaforge/envs/brain/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TruncatedSVD from version 1.0.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/lucascamillo/mambaforge/envs/brain/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator 

## Load weights into base model

In [7]:
weights = torch.tensor(ard_model.coef_).unsqueeze(0).float()
intercept = torch.tensor([ard_model.intercept_]).float()
rotation = torch.tensor(dim_reduction.components_.T).float()
center = torch.tensor(0)

#### PC linear model

In [8]:
base_model = pya.models.PCLinearModel(input_dim=len(model.features), pc_dim=rotation.shape[1])

base_model.center.data = center.float()
base_model.rotation.data = rotation.float()
base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = None

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'de Lima Camillo, Lucas Paulo, et al. "Histone mark age of human '
             'tissues and cells." bioRxiv (2023): 2023-08.',
 'clock_name': 'camilloh3k27me3',
 'data_type': 'histone mark',
 'doi': 'https://doi.org/10.1101/2023.08.21.554165',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2023}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['ENSG00000230368', 'ENSG00000234711', 'ENSG00000283040', 'ENSG00000272438', 'ENSG00000230699', 'ENSG00000223764', 'ENSG00000187583', 'ENSG00000187642', 'ENSG00000272512', 'ENSG00000188290', 'ENSG00000187608', 'ENSG00000188157', 'LRG_198', 'ENSG00000217801', 'ENSG00000273443', 'ENSG00000237330', 'ENSG00000223823', 'ENSG00000207730', 'ENS

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 39.9888],
        [ 30.3743],
        [101.3877],
        [ 41.8950],
        [ 24.2335],
        [ 51.2079],
        [ 49.4518],
        [ 53.4217],
        [ 38.0464],
        [ 33.4102]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: HistoneClocks
